In [1]:
from langchain import SerpAPIWrapper, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, Tool, initialize_agent

llm = OpenAI(temperature=0.7)

tool_names = ["serpapi", "llm-math"]

tools = load_tools(tool_names, llm=llm)

search = SerpAPIWrapper()

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# tools = [
#     Tool(
#         name = "Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events"
#     ),
#     Tool(
#         name="Calculator",
#         func=llm_math_chain.run,
#         description="useful for when you need to answer questions about math"
#     )
# ]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

/Users/ken/.pyenv/versions/3.9.7/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [2]:
llm_math_chain.run("2*2")



> Entering new LLMMathChain chain...
2*2```text
2 * 2
```
...numexpr.evaluate("2 * 2")...

Answer: 4
> Finished chain.


'Answer: 4'

In [4]:
agent.run("最新のドル円の為替レートを2乗してください")



> Entering new AgentExecutor chain...
 為替レートを調べる必要がある
Action: Search
Action Input: 最新のドル円の為替レート
Observation: 【NQNニューヨーク=川上純平】15日のニューヨーク外国為替市場で円相場は横ばいだった。前日と同じ1ドル=145円50～60銭で取引を終えた。米連邦準備理事会（FRB）の金融 ...
Thought: 2乗して求める必要がある
Action: Calculator
Action Input: 145.50^2
Observation: Answer: 21170.25
Thought: 答えが出た
Final Answer: 21170.25

> Finished chain.


'21170.25'

In [9]:
llm_math_chain.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [14]:
agent.run

<bound method Chain.run of AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=['zero-shot-react-description'], metadata=None, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Act

In [26]:
agent.agent.dict()

{'llm_chain': {'memory': None,
  'verbose': False,
  'tags': None,
  'metadata': None,
  'prompt': {'input_variables': ['input', 'agent_scratchpad'],
   'output_parser': None,
   'partial_variables': {},
   'template': 'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scr

In [27]:
agent.agent.llm_prefix

'Thought:'

In [22]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'